In [8]:
import pandas as pd
import os
from landcover_assignment.transition_matrix import TransitionMatrix
from landcover_assignment.landcover import LandCover
from landcover_assignment.afforestation import Afforestation
import os

# Landcover Assignment Example

The landcover assignment module takes scenario inputs and inputs generated from the [grassland_production](https://colmduff.github.io/grassland_production/html/index.html) module and distributes the spared area to various land uses. It considers soil type in the process (avaialbility of mineral or organic soils). It also considers soil groups in relation to the availability of organic soils and the forest yield class. 

It produces the land use change dataframe for current and future land uses, as well as a transition matrix, and an afforestation dataframe that will then be used by the [cbm_runner](https://colmduff.github.io/cbm_runner/html/index.html) module

## Set UP

In [9]:
input_path = "../tests/data/"

baseline = 2020 #the calibration year
target_year = 2050 #the target year

scenario_dataframe = pd.read_csv(os.path.join(input_path, "scenario_dataframe.csv"), index_col=0) #scenario inputs 

spared_area = pd.read_csv(os.path.join(input_path, "spared_area.csv"), index_col=0) #total spared area 
spared_area_breakdown = pd.read_csv(os.path.join(input_path, "spared_area_breakdown.csv"), index_col=0) #spared area by soil group
grassland_area = pd.read_csv(os.path.join(input_path, "total_grassland_area.csv"), index_col=0) #total grassland area remaining


## Classes 

The ```TransitionMatrix``` class wil produce the transition matrix. While the ```LandCover``` class produces the land use change dataframe, including the proportions of various soil types and other environmental factors. 

Finally, the ```Afforestation``` class produces the afforested area by yield class for use in the ```cbm_runner``` module. 

In [10]:
transition = TransitionMatrix(
    baseline, target_year, scenario_dataframe, grassland_area, spared_area, spared_area_breakdown
)

land = LandCover(
    baseline, target_year, scenario_dataframe, grassland_area, spared_area, spared_area_breakdown
)

affor = Afforestation(baseline, target_year, scenario_dataframe, transition.create_transition_matrix())

### Combined land use dataframe 

In [11]:
print(land.combined_future_land_use_area())

    farm_id  year            land_use       area_ha  share_mineral  \
0     -2020  2020           grassland  3.904877e+06       0.919446   
1     -2020  2020             wetland  1.225774e+06       0.000231   
2     -2020  2020            cropland  7.457409e+05       1.000000   
3     -2020  2020              forest  7.789881e+05       0.682922   
4     -2020  2020          settlement  1.257917e+05       0.975600   
5     -2020  2020  farmable_condition  0.000000e+00       1.000000   
6         0  2050           grassland  1.071079e+06       0.732777   
7         0  2050             wetland  1.254112e+06       0.000226   
8         0  2050            cropland  7.457409e+05       1.000000   
9         0  2050              forest  2.479267e+06       0.900374   
10        0  2050          settlement  1.257917e+05       0.975600   
11        0  2050  farmable_condition  1.105181e+06       1.000000   
12        1  2050           grassland  1.058037e+06       1.000000   
13        1  2050   

### Transition Matrix

In [12]:
print(transition.create_transition_matrix())

       Grassland_to_Grassland  Grassland_to_Wetland  Grassland_to_Cropland  \
-2020            0.000000e+00              0.000000                    0.0   
 0              -2.833798e+06          28337.977911                    0.0   
 1              -2.846840e+06         314554.726543                    0.0   

       Grassland_to_Forest  Grassland_to_Settlement  \
-2020         0.000000e+00                      0.0   
 0            1.700279e+06                      0.0   
 1            2.532285e+06                      0.0   

       Grassland_to_Farmable_Condition  Wetland_to_Grassland  \
-2020                     0.000000e+00                   0.0   
 0                        1.105181e+06                   0.0   
 1                        0.000000e+00                   0.0   

       Wetland_to_Wetland  Wetland_to_Cropland  Wetland_to_Forest  ...  \
-2020                 0.0                  0.0                0.0  ...   
 0                    0.0                  0.0               

### Afforestation Dataframe

In [13]:
print(affor.gen_cbm_afforestation_dataframe(spared_area_breakdown))

    scenario species yield_class     total_area
0         -1   Sitka     YC17_20       0.000000
1         -1   Sitka     YC20_24       0.000000
2         -1   Sitka     YC24_30       0.000000
3         -1     SGB         YC6       0.000000
4         -1     SGB         YC6       0.000000
5         -1     SGB         YC6       0.000000
6          0   Sitka     YC24_30  417595.058325
7          0     SGB         YC6  715935.503074
8          0   Sitka     YC20_24  208791.204651
9          0     SGB         YC6  357956.908635
10         0   Sitka     YC17_20       0.000000
11         0     SGB         YC6       0.000000
12         1   Sitka     YC24_30  418877.449812
13         1     SGB         YC6  718134.067390
14         1   Sitka     YC20_24  508198.559238
15         1     SGB         YC6  871268.430782
16         1   Sitka     YC17_20    5823.216747
17         1     SGB         YC6    9983.469700
